## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-10-53-27 +0000,bbc,Tourist plane with 12 on board crashes near Ke...,https://www.bbc.com/news/articles/c75146k9z4no...
1,2025-10-28-10-47-16 +0000,nyt,Sudan’s Military Withdraws From El Fasher in D...,https://www.nytimes.com/2025/10/28/world/afric...
2,2025-10-28-10-46-14 +0000,nyt,"In a Looming Nuclear Arms Race, Aging Los Alam...",https://www.nytimes.com/2025/10/28/us/los-alam...
3,2025-10-28-10-45-00 +0000,wsj,"Amazon Lays Off 14,000 Corporate Workers",https://www.wsj.com/tech/amazon-to-layoff-tens...
4,2025-10-28-10-43-00 +0000,wsj,China Vows to Significantly Boost Household Co...,https://www.wsj.com/world/china/china-vows-to-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,47
88,new,20
30,china,17
340,he,16
214,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
81,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,105
150,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,102
244,2025-10-27-17-47-39 +0000,nypost,Trump reveals he was ‘very much opposed’ to ma...,https://nypost.com/2025/10/27/business/trump-s...,96
322,2025-10-27-12-47-34 +0000,cbc,When will Trump's tariff increase hit Canada? ...,https://www.cbc.ca/news/world/trump-tariff-can...,95
296,2025-10-27-14-45-03 +0000,wapo,"Trump rebukes Putin for testing new missiles, ...",https://www.washingtonpost.com/world/2025/10/2...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
81,105,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...
8,65,2025-10-28-10-34-32 +0000,nypost,Menacing video shows ‘monster eye’ of Hurrican...,https://nypost.com/2025/10/28/world-news/menac...
148,64,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
150,55,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...
269,45,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
240,42,2025-10-27-18-01-17 +0000,nypost,Andrew Cuomo narrows gap against Zohran Mamdan...,https://nypost.com/2025/10/27/us-news/andrew-c...
129,40,2025-10-28-01-00-00 +0000,wsj,Watering down dish soap. Trading down to groce...,https://www.wsj.com/economy/consumers/the-leng...
58,39,2025-10-28-09-00-00 +0000,nypost,Biden White House doctor Kevin O’Connor referr...,https://nypost.com/2025/10/28/us-news/biden-wh...
327,37,2025-10-27-12-01-00 +0000,wsj,Milei Wins Mandate for Free-Market Revolution ...,https://www.wsj.com/world/americas/milei-wins-...
267,34,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
